In [1]:
!pip install seaborn==0.10
!pip install missingno

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import numpy as np
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


pd.options.display.max_columns=300

In [3]:
max_year='2020'
max_month='12'

prefix = max_year + '-' + max_month

filename = 'empresas_cluster.csv.gz'

cols_not=['RAZON_SOCIAL','CIIU','CIIU_V4','SEGMENTACION','GRUPOECONOMICO',
         'PATRIMONIO 2019','DIRECCION','FACEBOOK','PAGWEB','EMAIL_CORP',
         'PROVINCIA']


cols_yes=['RUC','TIEMPO_MERCADO','SUBSECTOR_TE','FACTURA USD'
          ,'NUM_ENT_SBS','CAL_GRAL','TIPO_CRED',
         'CRED_TOTAL'
          ,'EDAD_PROM_TRAB','Q1_EDAD','Q2_EDAD','Q3_EDAD','INGRESO_PROM_TRAB',
         'Q1_INGRESO','Q2_INGRESO','Q3_INGRESO','NRO_TRABAJADORES','DISTRITO','CONO_AGRUP_NUEVO']

col_rcc=['NUM_ENT_SBS','CAL_GRAL','TIPO_CRED','CRED_TOTAL']


var_num=['TIEMPO_MERCADO','FACTURA USD','NUM_ENT_SBS','CRED_TOTAL','EDAD_PROM_TRAB',
         'INGRESO_PROM_TRAB','NRO_TRABAJADORES']

var_cat=['SUBSECTOR_TE','CAL_GRAL','TIPO_CRED','DISTRITO','CONO_AGRUP_NUEVO']

col_cluster=['TIEMPO_MERCADO','FACTURA USD'
            ,'CRED_TOTAL'
             ,'Q1_INGRESO'
            ,'Q3_INGRESO'
            ,'NRO_TRABAJADORES'
             ,'ENC_CAL_TIPO'
             ,'ENC_CONO_AGRUP_NUEVO'
            ]

n_clusters=5



In [4]:
bucket='s3://rimac-analytics-temporal/'
path_input=bucket + 'individuals/Compartido/Andres Morita/201812 Segmentación Empresas/1. Inputs/'

path_tablas='s3://rimac-analytics-temporal/individuals/Stev/project_mse/mse_inputs/'

path_empresas=path_tablas + 'empresas_mercado_{}.csv.gz'.format(prefix)



In [5]:
path_empresas

's3://rimac-analytics-temporal/individuals/Stev/project_mse/mse_inputs/empresas_mercado_2020-12.csv.gz'

In [6]:
def quantile_1(serie):
    return serie.quantile(.25)

def quantile_2(serie):
    return serie.quantile(.5)

def quantile_3(serie):
    return serie.quantile(.75)

def mode(serie):
    return serie.mode()[0]


def preprocessing_data(df_empresas):
    df_empresas=df_empresas[[col for col in df_empresas.columns if 'COT'not in col]]
    
    print('Data before fillna:')
    print(df_empresas[cols_yes].dropna().shape)
    
    df_empresas['NUM_ENT_SBS']=df_empresas['NUM_ENT_SBS'].fillna(0)

    df_empresas['CAL_GRAL']=df_empresas['CAL_GRAL'].fillna('OTR')

    df_empresas['TIPO_CRED']=df_empresas['TIPO_CRED'].fillna('OTRO')

    df_empresas['CRED_TOTAL']=df_empresas['CRED_TOTAL'].fillna(0)
    
    print('Data after fillna:')
    print(df_empresas[cols_yes].dropna().shape)
    
    return df_empresas
   
    
    
def processingdata_to_clustering(df_empresas_notna):
    
    print("Processing categorical vars")
    #cal_gral
    df_empresas_notna.loc[df_empresas_notna.CAL_GRAL=='OK','ENC_CAL_GRAL']=5
    df_empresas_notna.loc[df_empresas_notna.CAL_GRAL=='CPP','ENC_CAL_GRAL']=4
    df_empresas_notna.loc[df_empresas_notna.CAL_GRAL=='DEF','ENC_CAL_GRAL']=3
    df_empresas_notna.loc[df_empresas_notna.CAL_GRAL=='DUD','ENC_CAL_GRAL']=2
    df_empresas_notna.loc[df_empresas_notna.CAL_GRAL=='PER','ENC_CAL_GRAL']=1
    df_empresas_notna.loc[df_empresas_notna.CAL_GRAL=='OTR','ENC_CAL_GRAL']=0
    
    #tipo_Cred
    df_empresas_notna.loc[df_empresas_notna.TIPO_CRED=='CORPORATIVO','ENC_TIPO_CRED']=5
    df_empresas_notna.loc[df_empresas_notna.TIPO_CRED=='GRANDESEMP','ENC_TIPO_CRED']=4
    df_empresas_notna.loc[df_empresas_notna.TIPO_CRED=='MEDIANASEMP','ENC_TIPO_CRED']=3
    df_empresas_notna.loc[df_empresas_notna.TIPO_CRED=='PEQUEÑASEMP','ENC_TIPO_CRED']=2
    df_empresas_notna.loc[df_empresas_notna.TIPO_CRED=='MICROEMP','ENC_TIPO_CRED']=1
    df_empresas_notna.loc[df_empresas_notna.TIPO_CRED=='OTRO','ENC_TIPO_CRED']=0
    
    df_empresas_notna['ENC_CAL_TIPO']=(df_empresas_notna['ENC_CAL_GRAL']+df_empresas_notna['ENC_TIPO_CRED'])/2
    
    #sectoreconom
    df_empresas_notna['ENC_SUBSECTOR_TE']=df_empresas_notna['SUBSECTOR_TE'].map(df_empresas_notna['SUBSECTOR_TE'].value_counts()/len(df_empresas_notna))
    
    #CONO_AGRUP_NUEVO
    df_empresas_notna['ENC_CONO_AGRUP_NUEVO']=df_empresas_notna['CONO_AGRUP_NUEVO'].map(df_empresas_notna['CONO_AGRUP_NUEVO'].value_counts()/len(df_empresas_notna))

    le=LabelEncoder()
    df_empresas_notna['LE_CONO_AGRUP_NUEVO']=le.fit_transform(df_empresas_notna['CONO_AGRUP_NUEVO'])
    
    
    print("Processing numerical vars")
    
    df_empresas_notna['LOG_FACTURA USD']=np.log(df_empresas_notna['FACTURA USD'])
    df_empresas_notna['LOG_CRED_TOTAL']=np.log(df_empresas_notna['CRED_TOTAL']+1)
    df_empresas_notna['LOG_TIEMPO_MERCADO']=np.log(df_empresas_notna['TIEMPO_MERCADO']+1)
    df_empresas_notna['LOG_NRO_TRABAJADORES']=np.log(df_empresas_notna['NRO_TRABAJADORES'])
    df_empresas_notna['LOG_INGRESO_PROM_TRAB']=np.log(df_empresas_notna['INGRESO_PROM_TRAB'])
    df_empresas_notna['LOG_Q1_INGRESO']=np.log(df_empresas_notna['Q1_INGRESO'])
    df_empresas_notna['LOG_Q2_INGRESO']=np.log(df_empresas_notna['Q2_INGRESO'])
    df_empresas_notna['LOG_Q3_INGRESO']=np.log(df_empresas_notna['Q3_INGRESO'])
    df_empresas_notna['RIQ']=df_empresas_notna['Q3_INGRESO']-df_empresas_notna['Q1_INGRESO']
    
    
    return df_empresas_notna



def get_clustering(df_empresas_notna, df_empresas):
    
    print("PCA with 3 components")
    
    X=df_empresas_notna[col_cluster]
    X=StandardScaler().fit_transform(X)

    pca=PCA(n_components=3)
    principal_components=pca.fit_transform(X)
    
    columns_pca=[ 'pc_' + str(i + 1) for i in range(len(principal_components.T))]
    df_pc=pd.DataFrame(data=principal_components, columns=columns_pca)
    n_components=len(principal_components.T)
    
    print("KMeans with {} cluster".format(n_clusters))
    
    model=KMeans(random_state=0,n_jobs=-1,n_clusters=n_clusters)
    kmean_model=model.fit(df_pc)
    labels=kmean_model.labels_
    df_pc['cluster']=kmean_model.labels_
    
    df_pc['RUC']=df_empresas_notna['RUC'].reset_index(drop=True)
    
    df_empresas_cluster=df_empresas.merge(df_pc[['RUC','cluster']],how='left',on='RUC')
    
    print("Shape of data with clusters")
    print(df_empresas_cluster.shape)
    
    print('N-unique ruc {}'.format(df_empresas_cluster.RUC.nunique()))
    
    
    return df_empresas_cluster

In [7]:
def clustering_data(path_empresas):
    df_empresas=pd.read_csv(path_empresas,encoding='latin1',dtype={'RUC':str})
    print(df_empresas.shape)
    
    df_empresas = preprocessing_data(df_empresas)
    
    df_empresas_notna=df_empresas[cols_yes].dropna()
    
    df_empresas_notna = processingdata_to_clustering(df_empresas_notna)
    
    df_empresas_cluster = get_clustering(df_empresas_notna, df_empresas)
    
    
    return df_empresas_cluster

In [8]:
if __name__ == '__main__':
    df_empresas_cluster = clustering_data(path_empresas)
    
    print('Saving file in local, the name file is {}'.format(filename))
    df_empresas_cluster[['RUC','cluster']].to_csv(filename,index=False,compression='gzip')


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (13,17) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(661878, 70)
Data before fillna:
(15100, 19)
Data after fillna:


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


(23535, 19)
Processing categorical vars
Processing numerical vars
PCA with 3 components
KMeans with 5 cluster
Shape of data with clusters
(661878, 48)
N-unique ruc 661878
Saving file in local, the name file is empresas_cluster.csv.gz


In [9]:
df_empresas_cluster.cluster.notna().sum()

23535

In [9]:
df_empresas_cluster.cluster.notna().sum()

46026

In [10]:
df_empresas_cluster.groupby('cluster').agg({'RUC':'count',
                                        'FACTURA USD':['mean',quantile_2,'min','max'],
                                        'CRED_TOTAL':['mean',quantile_2],
                                       'NRO_TRABAJADORES':['mean',quantile_2],
                                        'Q2_INGRESO':['mean',quantile_2],
                                       'TIEMPO_MERCADO':['mean',quantile_2],
                                        'CAL_GRAL':[mode],
                                       'CONO_AGRUP_NUEVO':[mode]})

RUC   FACTURA USD                                            \
         count          mean    quantile_2           min           max   
cluster                                                                  
0.0        989  3.414356e+07  3.240036e+06  6.437768e+03  1.394889e+09   
1.0      10095  9.942079e+06  1.673258e+06  3.209289e+04  8.122908e+08   
2.0         52  1.282667e+09  1.086097e+09  8.518406e+07  4.307880e+09   
3.0         75  2.296656e+07  2.905384e+06  2.281715e+05  4.533009e+08   
4.0      12324  2.502683e+06  7.664011e+05  4.171367e+04  2.903864e+08   

           CRED_TOTAL               NRO_TRABAJADORES               Q2_INGRESO  \
                 mean    quantile_2             mean quantile_2          mean   
cluster                                                                         
0.0      8.558954e+06  0.000000e+00        35.083923        7.0   9340.668352   
1.0      6.047077e+06  4.700684e+05        83.446855       13.0   1750.401833   
2.0      4.435881e+08  2.316596e+08      4134.173077     3066.5   4709.245192   
3.0      1.175707e+07  0.000000e+00         7.853333        2.0  41809.123333   
4.0      5.362879e+05  0.000000e+00        20.236368        6.0   1669.349420   

                   TIEMPO_MERCADO            CAL_GRAL CONO_AGRUP_NUEVO  
        quantile_2           mean quantile_2     mode             mode  
cluster                                                                 
0.0        7846.00      14.783620       13.0      OTR         LIMA TOP  
1.0        1520.50      18.379891       19.0       OK      LIMA CENTRO  
2.0        3468.75      29.076923       26.0       OK         LIMA TOP  
3.0       33737.00      14.066667       13.0      OTR         LIMA TOP  
4.0        1393.75      10.771016       10.0      OTR        PROVINCIA

In [10]:
df_empresas_cluster.groupby('cluster').agg({'RUC':'count',
                                        'FACTURA USD':['mean',quantile_2,'min','max'],
                                        'CRED_TOTAL':['mean',quantile_2],
                                       'NRO_TRABAJADORES':['mean',quantile_2],
                                        'Q2_INGRESO':['mean',quantile_2],
                                       'TIEMPO_MERCADO':['mean',quantile_2],
                                        'CAL_GRAL':[mode],
                                       'CONO_AGRUP_NUEVO':[mode]})

RUC   FACTURA USD                                            \
         count          mean    quantile_2           min           max   
cluster                                                                  
0.0      24089  1.637139e+06  7.605720e+05  1.532802e+04  2.087009e+08   
1.0        137  1.325187e+07  1.072962e+06  9.809933e+03  6.693033e+08   
2.0       2460  1.376878e+07  1.064800e+06  6.437768e+03  1.086097e+09   
3.0         69  1.010414e+09  8.122908e+08  7.757834e+07  4.307880e+09   
4.0      19271  5.393429e+06  1.064800e+06  9.106990e+03  1.086097e+09   

           CRED_TOTAL               NRO_TRABAJADORES_2019             \
                 mean    quantile_2                  mean quantile_2   
cluster                                                                
0.0      3.117031e+05  0.000000e+00             19.840965        6.0   
1.0      4.551281e+06  0.000000e+00              7.036496        2.0   
2.0      3.544253e+06  0.000000e+00             28.063821        5.0   
3.0      3.882815e+08  2.180393e+08           4369.666667     3158.0   
4.0      3.241145e+06  3.095343e+05             68.658710       12.0   

           Q2_INGRESO               TIEMPO_MERCADO            CAL_GRAL  \
                 mean    quantile_2           mean quantile_2     mode   
cluster                                                                  
0.0       1543.220360   1249.701600       9.839387        9.0      OTR   
1.0      30661.449660  26624.371879      12.861314       11.0      OTR   
2.0       7352.112413   6442.211748      13.064228       11.0      OTR   
3.0       4052.564949   3375.235738      25.478261       24.0       OK   
4.0       1623.194773   1374.671760      17.862228       17.0       OK   

        CONO_AGRUP_NUEVO  
                    mode  
cluster                   
0.0            PROVINCIA  
1.0             LIMA TOP  
2.0             LIMA TOP  
3.0             LIMA TOP  
4.0          LIMA CENTRO